In [1]:
%pip install langchain-core langgraph>0.2.27


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["LANGSMITH"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [3]:
if not os.environ.get("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass.getpass("Enter API key for gemini")

from langchain.chat_models import init_chat_model
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, I am Aarshita")])

AIMessage(content="Hi Aarshita! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ae096f76-ee4e-44d2-9369-25c955e92ac2-0', usage_metadata={'input_tokens': 7, 'output_tokens': 21, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}})

trying to check memory, since it does not have any concept of state for now

In [5]:
model.invoke([HumanMessage(content="What is my name?")])

AIMessage(content='I am an AI language model, and I do not have access to personal information about you. Therefore, I do not know your name.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f5e8996d-7248-4739-9852-7e723247dbe4-0', usage_metadata={'input_tokens': 5, 'output_tokens': 29, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}})

Now, let us pass to entire conversation history to see what happens

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Aarshita"),
        AIMessage(content="Hello Aarshita! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Aarshita.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ea84c119-81d5-4e96-b7f8-10f982375525-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}})

So, we need memory persistence. We use LangGraph for this

In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = model.invoke(state['messages'])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [8]:
config = {"configurable": {"thread_id": "abc123"}}

In [11]:
query = "Hi! I am Aarshita"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Aarshita! It's lovely to meet you. What can I do for you today?


In [12]:
query = "What is my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Aarshita.


perfect, now chatbot remembers details, if we change thread, it starts a fresh conversation

In [13]:
config = {"configurable": {"thread_id": "abc234"}}
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I am an AI assistant, and I don't know your name. You have not told me your name.


we can always go back to old conversations

In [14]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As you told me earlier, your name is Aarshita.
